In [1]:
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.data import HeteroData

In [2]:
dataset = TUDataset(root='data/ENZYMES', name='ENZYMES')

In [4]:
c_list = []
for idx, datum in enumerate(dataset):
    if 1.0 in datum.x.T[-1]:
        c_list.append(idx)
    

In [5]:
c_list

[10,
 61,
 66,
 79,
 115,
 119,
 121,
 123,
 124,
 125,
 130,
 131,
 132,
 133,
 138,
 144,
 161,
 163,
 200,
 214,
 215,
 254,
 255,
 256,
 259,
 272,
 442,
 443,
 447,
 450,
 452,
 458,
 459,
 460,
 463,
 464,
 523,
 540,
 542,
 543,
 544,
 545,
 552]

In [6]:
all_list = []
for idx, datum in enumerate(dataset):
    if 1.0 in datum.x.T[2] and 1.0 in datum.x.T[1] and 1.0 in datum.x.T[0]:
        all_list.append(idx)

In [7]:
all_list

[61,
 66,
 79,
 115,
 119,
 121,
 123,
 124,
 125,
 130,
 131,
 132,
 133,
 138,
 144,
 161,
 163,
 200,
 214,
 215,
 254,
 255,
 256,
 259,
 272,
 442,
 443,
 447,
 450,
 452,
 458,
 459,
 460,
 463,
 464,
 523,
 540,
 542,
 543,
 544,
 545,
 552]